In [1]:
from FR3Py.robot.model import PinocchioModel
from liegroups import SO3
import numpy as np

In [2]:
from FR3Py.robot.interface import FR3Real

In [3]:
robot = FR3Real(robot_id='fr3')

Interface Running...


In [4]:
model = PinocchioModel()

In [5]:
robot.getStates()

{'q': array([ 8.73126516e-04, -7.84232008e-01,  3.48465278e-04, -2.45623620e+00,
        -1.39591939e-03,  1.57099680e+00,  7.86290757e-01,  0.00000000e+00,
         0.00000000e+00]),
 'dq': array([-5.99765444e-04, -3.67203978e-04,  5.26246868e-04, -4.50767107e-04,
        -1.78947893e-04, -8.67564905e-05, -3.34819662e-04,  0.00000000e+00,
         0.00000000e+00]),
 'T': array([-0.1355927 , -4.52417135, -0.79939878, 23.42898941,  1.05577314,
         2.02237678, -0.08283016,  0.        ,  0.        ])}

In [8]:
import time
state = robot.getStates()
info = model.getInfo(state['q'], state['dq'])
R0 = info['R_EE']
p0 = info['P_EE'].reshape(3,1)

Kv = 3*np.eye(3)
Kp = 20*np.eye(3)
while True:
    time.sleep(0.002)
    state = robot.getStates()
    info = model.getInfo(state['q'], state['dq'])
    J = info['J_EE'][:, :7]
    Jr = J[3:, :] 
    Jp = J[:3, :]
    vel = (Jp@state['dq'][:7].reshape(7,1))
    omega = (Jr@state['dq'][:7]).reshape(3,1)

    rot_error = SO3(R0@info['R_EE'].T).log().reshape(3,1)

    task_error = 10.*(np.zeros(3).reshape(3,1) - vel)+15*(p0-info['P_EE'].reshape(3,1))
    task_error[0:2] = 0.
    task_error[2] = -16.
    
    tau = Jr.T@(Kp@rot_error - Kv@omega) + Jp.T@(task_error)
    robot.sendCommands(np.hstack([tau.squeeze(), np.zeros(2)]))
    # print(tau.T)

TypeError: 'NoneType' object is not subscriptable

In [57]:
p0.reshape(3,1)

array([[0.48163057],
       [0.00371456],
       [0.00278134]])